IMDb is one of the largest internet database for movies. It contains a lot of information regarding a film, be it the casts, crews, and also reviews from the watchers.

In this notebook, we are going to explore and analyze the IMDb data downloaded from here https://www.kaggle.com/datasets/luizpaulodeoliveira/imdb-project-sql?resource=download 

The database consists of two main tables, movies and directors.

First we are going to setup and connect to the local database.

In [1]:
%load_ext sql
%config SqlMagic.autocommit=False

In [2]:
%sql sqlite:///imdb.sqlite

Check the data from each tables

In [3]:
%%sql

SELECT * FROM movies
LIMIT 5

 * sqlite:///imdb.sqlite
Done.


id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",Enter the World of Pandora.,19995,4762
43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.","At the end of the world, the adventure begins.",285,4763
43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",A Plan No One Escapes,206647,4764
43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.",The Legend Ends,49026,4765
43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands.","Lost in our world, found in another.",49529,4766


In [4]:
%%sql

SELECT * FROM directors
LIMIT 5

 * sqlite:///imdb.sqlite
Done.


name,id,gender,uid,department
James Cameron,4762,2,2710,Directing
Gore Verbinski,4763,2,1704,Directing
Sam Mendes,4764,2,39,Directing
Christopher Nolan,4765,2,525,Directing
Andrew Stanton,4766,2,7,Directing


Most popular movies each year

In [5]:
%%sql

SELECT substr(release_date,1,4) as Year, Title
FROM movies
GROUP BY Year
HAVING max(popularity)
ORDER BY Year DESC
LIMIT 15

 * sqlite:///imdb.sqlite
Done.


Year,title
2016,Deadpool
2015,Minions
2014,Interstellar
2013,Frozen
2012,The Avengers
2011,Rise of the Planet of the Apes
2010,Inception
2009,Avatar
2008,The Dark Knight
2007,Pirates of the Caribbean: At World's End


Top 5 most productive directors

In [6]:
%%sql

SELECT directors.name, count(*) as num_of_movies
FROM movies, directors
WHERE directors.id=movies.director_id
GROUP BY directors.name
ORDER BY count(*) desc
LIMIT 10

 * sqlite:///imdb.sqlite
Done.


name,num_of_movies
Steven Spielberg,27
Woody Allen,21
Martin Scorsese,20
Clint Eastwood,20
Spike Lee,16
Robert Rodriguez,16
Ridley Scott,16
Steven Soderbergh,15
Renny Harlin,15
Tim Burton,14


Now we are going to see if month of released date contribute to revenue factor

In [7]:
%%sql

SELECT substr(release_date,6,2) as Month, sum(revenue) as Monthly_Revenue, count(*) as Num_of_Movies, round(sum(revenue)/count(*)) as Average_Revenue
FROM movies
GROUP BY Month


 * sqlite:///imdb.sqlite
Done.


Month,Monthly_Revenue,Num_of_Movies,Average_Revenue
01,10938212188,372,29403796.0
02,18301124650,324,56484952.0
03,27194373719,370,73498307.0
04,25874666986,350,73927619.0
05,48616638167,361,134672127.0
06,59238687315,389,152284543.0
07,40502377467,360,112506604.0
08,23047363225,412,55940202.0
09,23513601546,585,40194190.0
10,25071742760,472,53118099.0


Based on month of release_date, movies tend to get far more revenue in June (averaged), it's more than 5 times of revenue movies released in January, but there are a lot of factors for revenue.
Also, note that September leads with the highest number of movies released.

Here are the top 5 highest grossing directors.

In [8]:
%%sql

SELECT DISTINCT directors.name, sum(movies.revenue) over (partition by directors.name) as total_revenue
FROM movies, directors
WHERE directors.id=movies.director_id
ORDER BY total_revenue DESC
LIMIT 5

 * sqlite:///imdb.sqlite
Done.


name,total_revenue
Steven Spielberg,9147393164
Peter Jackson,6498642820
James Cameron,5883569439
Michael Bay,5832524638
Christopher Nolan,4227483234


Take a look at how top 5 highest grossing director choose their release date.

In [9]:
%%sql

SELECT DISTINCT substr(release_date,6,2) as Month, count(title) over (partition by substr(release_date,6,2)) as count_m
FROM directors
INNER JOIN movies ON movies.director_id=directors.id
WHERE directors.name IN(
	SELECT sub.name FROM(
		SELECT DISTINCT directors.name, sum(movies.revenue) over (partition by directors.name) as total_revenue
		FROM movies, directors
		WHERE directors.id=movies.director_id
		ORDER BY total_revenue desc) sub
	LIMIT 5)
ORDER BY month

 * sqlite:///imdb.sqlite
Done.


Month,count_m
01,1
04,3
05,6
06,15
07,10
08,1
09,1
10,5
11,6
12,15


Looking at the number, the range is quite wide. Indeed June is one of favorite month to release their movies along with December.

To get a higher revenue, should the movie has a high budget? Is there any correlation between movie popularity, revenue, and quality?

In [10]:
%%sql

SELECT sub.percentile_rank, round(avg(sub.revenue),2) as average_revenue, 
    round(avg(sub.popularity),2) as average_popularity, round(avg(sub.vote_average),2) as average_voting
FROM (SELECT BUDGET, REVENUE, popularity, vote_average, 
      ROUND(PERCENT_RANK() OVER (ORDER BY budget) ,1) as percentile_rank
		FROM movies) sub
GROUP BY sub.percentile_rank

 * sqlite:///imdb.sqlite
Done.


percentile_rank,average_revenue,average_popularity,average_voting
0.0,3554574.41,4.82,5.67
0.2,9860865.75,8.65,6.01
0.3,19146851.04,13.64,6.3
0.4,34612372.51,15.96,6.38
0.5,50716068.62,19.09,6.26
0.6,58414990.01,20.67,6.16
0.7,85175005.38,24.67,6.21
0.8,118076956.55,29.26,6.11
0.9,207872026.03,38.37,6.17
1.0,509621727.33,75.81,6.39


We divided the budget into percentiles. 
Based on the data, indeed, there is a strong possitive correlation between budget and revenue. The possitive correlation could also be found between budget and popularity.
Voting from watchers though, seems not to be really affected by the budget.

Now that we know that revenue is linked to popularity, we are going to see is directors name important factor for a movie success. We are going to choose top 5 highest grossing directors and see if their name guarantee the movie success.

In [11]:
%%sql

SELECT directors.name, min(movies.vote_average) as min_voting, round(avg(movies.vote_average),2) as average_voting, 
    min(movies.popularity) as min_popularity, round(avg(movies.popularity),2) as average_popularity, 
    min(movies.revenue) as min_revenue, round(avg(movies.revenue)) as average_revenue
FROM directors
INNER JOIN movies ON movies.director_id=directors.id
WHERE directors.name in(
    SELECT sub.name from(
        SELECT DISTINCT directors.name, sum(movies.revenue) over (partition by directors.name) as total_revenue
        from movies, directors
        where directors.id=movies.director_id
        order by total_revenue desc) sub
    limit 5)
GROUP BY directors.name
ORDER BY sum(movies.revenue) desc

 * sqlite:///imdb.sqlite
Done.


name,min_voting,average_voting,min_popularity,average_popularity,min_revenue,average_revenue
Steven Spielberg,5.6,6.97,2,48.0,29450919,338792339.0
Peter Jackson,6.6,7.33,8,87.44,3049135,722071424.0
James Cameron,6.8,7.33,24,79.14,78371200,840509920.0
Michael Bay,5.8,6.4,21,42.83,69411370,486043720.0
Christopher Nolan,6.8,7.8,41,185.0,39723096,528435404.0


Based on the data above, it depends on the directors themselves. Some directors managed to keep getting high numbers on popularity and revenue, meanwhile some of them could not. 

We can conclude, based on this data, revenue seems to be affected by a few factors like budget, and popularity. Directors also play a huge part, but can not guarantee the movie success. Choosing the right time for release date (esp. month) might also help to maximize the revenue.